# NP models

In [1]:
# load packages
import xarray as xr
import numpy as np
import pandas as pd
import torch
import neuralprocesses.torch as nps
import plotly.graph_objects as go

/home/kim/smb_emulator-main/.venv/lib/python3.11/site-packages/fdm/fdm.py:38: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  coefs = mat.inv()[:, deriv] * np.math.factorial(deriv)
/home/kim/smb_emulator-main/.venv/lib/python3.11/site-packages/fdm/fdm.py:44: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  / np.math.factorial(order)
/home/kim/smb_emulator-main/.venv/lib/python3.11/site-packages/fdm/fdm.py:38: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  coefs = mat.inv()[:, deriv] * np.math.factorial(deriv)
/home/kim/smb_emulator-main/.venv/lib/python3.11/site-packages/fdm/fdm.py:44: DeprecationWarning: `np.math` is a deprecated alias for the standard library 

In [34]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device name : {device}")
print(torch.cuda.is_available())

print(torch.cuda.current_device()) 
print(torch.cuda.device_count()) # 1 GPU available
print(torch.cuda.get_device_name())
print(torch.cuda.get_device_properties(0)) # 26GB memory

Device name : cuda:0
True
0
1
NVIDIA GeForce RTX 4090
_CudaDeviceProperties(name='NVIDIA GeForce RTX 4090', major=8, minor=9, total_memory=24209MB, multi_processor_count=128)


In [35]:
path_MAR = "/home/kim/data/zenodo_7875882_MAR_ACCESS/MAR(ACCESS1-3)_monthly_SMB.nc"
path_ACCESS = "/home/kim/data/zenodo_7875882_MAR_ACCESS/ACCESS1-3-stereographic_monthly_cleaned.nc"

xr_mar = xr.open_dataset(path_MAR)
xr_access = xr.open_dataset(path_ACCESS)

print(f"Shape of MAR smb data: {xr_mar.SMB.shape}")
print(f"Shape of MAR smb data: {xr_access.huss.shape}")

Shape of MAR smb data: (1452, 148, 176)
Shape of MAR smb data: (1452, 25, 90)


Notes on data gridding:
- inputs (xr_access) and target (xr_mar) have different grid
- inputs should have a square grid for a CNN
- Can the model map from one to the other?
- Do we need to input a location grid?

In [4]:
xr_mar

<xarray.Dataset>
Dimensions:      (time: 1452, x: 176, y: 148)
Coordinates:
  * time         (time) datetime64[ns] 1980-01-31 1980-02-29 ... 2100-12-31
  * x            (x) float32 -3.01e+06 -2.975e+06 ... 3.08e+06 3.115e+06
  * y            (y) float32 -2.555e+06 -2.52e+06 ... 2.555e+06 2.59e+06
    spatial_ref  int64 ...
Data variables:
    SMB          (time, y, x) float32 ...

In [36]:
xr_access["x"]
xr_access["y"]

<xarray.DataArray 'y' (y: 25)>
array([-2472000.  , -2266000.  , -2060000.  , -1853999.9 , -1647999.9 ,
       -1442000.  , -1236000.  , -1030000.  ,  -823999.94,  -618000.  ,
        -411999.97,  -205999.98,        0.  ,   205999.98,   411999.97,
         618000.  ,   823999.94,  1030000.  ,  1236000.  ,  1442000.  ,
        1647999.9 ,  1853999.9 ,  2060000.  ,  2266000.  ,  2472000.  ],
      dtype=float32)
Coordinates:
    spatial_ref  int64 ...
  * y            (y) float32 -2.472e+06 -2.266e+06 ... 2.266e+06 2.472e+06

In [6]:
xr_access
# 9 variables + 2 seasonal variables

<xarray.Dataset>
Dimensions:      (time: 1452, x: 90, y: 25)
Coordinates:
    spatial_ref  int64 ...
  * time         (time) datetime64[ns] 1980-01-31 1980-02-29 ... 2100-12-31
  * x            (x) float32 -2.992e+06 -2.924e+06 ... 2.992e+06 3.06e+06
  * y            (y) float32 -2.472e+06 -2.266e+06 ... 2.266e+06 2.472e+06
Data variables:
    huss         (time, y, x) float32 ...
    hurs         (time, y, x) float32 ...
    pr           (time, y, x) float64 ...
    tas          (time, y, x) float32 ...
    ps           (time, y, x) float32 ...
    rsds         (time, y, x) float32 ...
    rlds         (time, y, x) float32 ...
    vas          (time, y, x) float32 ...
    uas          (time, y, x) float32 ...
Attributes: (12/29)
    institution:            CSIRO (Commonwealth Scientific and Industrial Res...
    institute_id:           CSIRO-BOM
    experiment_id:          historical
    source:                 ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-p...
    model_id:               ACCESS1.3
    forcing:                GHG, Oz, SA, Sl, Vl, BC, OC, (GHG = CO2, N2O, CH4...
    ...                     ...
    title:                  ACCESS1-3 model output prepared for CMIP5 historical
    parent_experiment:      pre-industrial control
    modeling_realm:         atmos
    realization:            1
    cmor_version:           2.8.0
    crs:                    3031

In [7]:
xr_mar

<xarray.Dataset>
Dimensions:      (time: 1452, x: 176, y: 148)
Coordinates:
  * time         (time) datetime64[ns] 1980-01-31 1980-02-29 ... 2100-12-31
  * x            (x) float32 -3.01e+06 -2.975e+06 ... 3.08e+06 3.115e+06
  * y            (y) float32 -2.555e+06 -2.52e+06 ... 2.555e+06 2.59e+06
    spatial_ref  int64 ...
Data variables:
    SMB          (time, y, x) float32 ...

In [80]:
# Shape: (c, n1, n2)
xr_mar.SMB.values.shape
# one time slide
xr_mar.SMB.values[0, :, :].shape
n_training_months = 80*12
n_val_months = 20*12

n_training_months = 1
n_val_months = 1


# convert to torch tensor and add explicit dimensionality
context_y_training = torch.tensor(xr_mar.SMB.values[0:n_training_months, :, :]).unsqueeze(1).cuda()
context_y_training.cuda()
# Assert
# context_y_training.is_cuda
context_y_training.shape

torch.Size([1, 1, 148, 176])

In [81]:
context_x_training = torch.tensor(np.transpose(xr_access.to_array().values, axes = (1, 0, 2, 3))[0:n_training_months, :, :, :])
context_x_training.cuda()

target_x = torch.tensor(np.transpose(xr_access.to_array().values, 
                                      axes = (1, 0, 2, 3))[n_training_months:(n_training_months + n_val_months), :, :, :])
target_x.cuda()

print(context_x_training.shape)
print(target_x.shape)
print(context_x_training.dtype)

torch.Size([1, 9, 25, 90])
torch.Size([1, 9, 25, 90])
torch.float64


Size of these tensors:   
(1, 9, 25, 90) = 20,250 (20k)  
torch.float64: 64 bits are 8 bytes  
8 * 1 * 9 * 25 * 90 = 162,000 bytes  
8 * 1 * 1 * 148 * 176 = 208,384 bytes (0.2 MB)  


## Seasonal encoding

- Encoding periodic/cyclical pattern of months: The fact that January (01) comes afer December (12) should be numerically presented.
- Why to we need cos and sin? Because both cosine and sine waves oscillate, two different months (e.g. September and March) are assigned the same cos() value. Thus we need an additional variable to differentiate these two.

In [48]:
# timestamp provided in numpy datatype datetime64
month_array = xr_mar.time.values.astype('datetime64[M]').astype(int) % 12 + 1
month_array.shape

cos_month = np.cos(2 * np.pi * month_array / 12)
sin_month = np.sin(2 * np.pi * month_array / 12)

# Min-Max normalise by subtracting min and dividing by range
year_array = xr_mar.time.values.astype('datetime64[Y]').astype(int) + 1970
# Use norm year as predictor too
year_norm = (year_array - np.min(year_array)) / (np.max(year_array) - np.min(year_array))

In [41]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = xr_mar.time.values.astype('datetime64[M]')[0:36], y = cos_month[0:36], 
                         mode = 'lines', name = 'cos_month'))
fig.add_trace(go.Scatter(x = xr_mar.time.values.astype('datetime64[M]')[0:36], y = sin_month[0:36], 
                         mode = 'lines', name = 'sin_month'))
fig.update_layout(title = "Cos Sine temporal encoding of months")
fig.show()

## Model

- shape of tensors: (b, c, n)
    - b: batch size
    - c: dimensionality/channels
    - n: number of data points
        - images: (b, c, *n) where n = (n1, n2)

In [127]:
convgnp = nps.construct_convgnp(dim_x = 3, dim_y = 1, likelihood = "het")
# model on GPU
# convgnp.to(device)

In [133]:
def extract_num_params(model):
    """Takes as input a torch model to output the number of parameters it has.

    Args:
        model (_type_): A torch model
    """
    n_params_list = []

    # Iterate through every layer
    for parameter in convgnp.parameters():
        # Multiply all shape dimensionalities of torch tensor
        n_params = torch.prod(torch.tensor(parameter.shape))
        # Add to list
        n_params_list.append(n_params.numpy().item())

    # Return sum
    return(sum(n_params_list), parameter.dtype)

n_params_model, dtype_last_layer = extract_num_params(convgnp)

print(f"Dtype used for last layer: {dtype_last_layer}")
print(f"Total number of parameters in model: {n_params_model}")
print(f"Bytes at float32: {n_params_model * 32}") 
print(f"GB at float32: {n_params_model * 32/1000000000}") 
# 0.067 GB (67 MB) for a GNP at this dimensionality
# 0.26 GB for ConvGNP

Dtype used for last layer: torch.float32
Total number of parameters in model: 8208900.0
Bytes at float32: 262684800.0
GB at float32: 0.2626848


In [144]:
# Construct/initialise model.
# We are using dim_x = 8 channels
# We are predicting a scalar output (smb): dim_y = 1

convgnp = nps.construct_climate_convgnp_multires()
# model on GPU
convgnp.to(device)

# Construct optimiser.
opt = torch.optim.Adam(convgnp.parameters(), 1e-3)

dist = convgnp(
   context_y_training,  # Context inputs
   context_y_training,  # Context outputs
   target_x,  # Target inputs
)

mean, var = dist.mean, dist.var  # Prediction for target outputs

print(dist.sample())
print(dist.kl(dist))
print(dist.entropy())

RuntimeError: Dispatched to fallback implementation for `code`, but specialised implementation are available. The arguments are `(RestructureParallel(), tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]], device='cuda:0'), tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]], device='cuda:0'), tensor([[[[ 5.4399e+00,  5.3600e+00,  5.2815e+00,  ...,  5.1299e+00,
            5.1974e+00,  5.2482e+00],
          [ 5.2639e+00,  5.1873e+00,  5.1121e+00,  ...,  4.9796e+00,
            5.0431e+00,  5.0620e+00],
          [ 5.0076e+00,  4.9331e+00,  4.8599e+00,  ...,  

The Kernel crashed while executing code in the the current cell or a previous cell. Please review the code in the cell(s) to identify a possible cause of the failure. Click here for more info. View Jupyter log for further details.

RuntimeError: [enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 332874448896 bytes. Error code 12 (Cannot allocate memory)

CUDA out of memory. Tried to allocate 310.01 GiB (GPU 0; 23.64 GiB total capacity; 12.58 GiB already allocated; 10.43 GiB free; 12.60 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid


GiB is roughly the same as GB.

64gb for roger

AttributeError: module 'torch.nn' has no attribute 'Conv4d': 3d conv are the max.
NotImplementedError: The FullConvGNP for now only supports single-dimensional inputs.